# Sentence Similarity

In [ ]:
''' Install libraries. '''
# !python3 -m pip install spacy

## Preprocess data

In [1]:
''' Data contains one sentence per line. '''
txt = open('dataset.txt', 'r', encoding='utf8').read().split('\n')
print('Number of sentences:', len(txt))
print(txt[1000:1002])

Number of sentences: 2534
['Über die Form kann inhaltlich Verschiedenes aufeinander bezogen werden, ohne die jeweilige Verschiedenheit aufzugeben.', 'Wenn zwei verschiedene Dinge auf dieselbe formale Struktur bezogen werden können, können sie sich auch aufeinander beziehen.']


In [2]:
''' Tokenize with pretrained word embeddings. '''
import spacy
# Download language model
# !python -m spacy download de_core_news_md
# https://spacy.io/models/de#de_core_news_md
spacy_de = spacy.load('de_core_news_md')
tokenized_txt = [spacy_de(sentence) for sentence in txt]

## Test

In [3]:
s1 = spacy_de('Maschine Algorithmus Software Möglichkeitsraum')
s2 = spacy_de('Doktor Fürst Hochgobernitz Gehirn')
s3 = spacy_de('Denken Gehirn Fürst')
s4 = spacy_de('System Maschine Theorie Implementierung')

In [4]:
print(s1.similarity(s2))
print(s1.similarity(s3))
print(s1.similarity(s4), '(Should be the highest score.)')
print('-'*10)
print(s2.similarity(s1))
print(s2.similarity(s3), '(Should be the highest score.)')
print(s2.similarity(s4))

0.14237512269583175
0.28613596500956817
0.8049022142081795 (Should be the highest score.)
----------
0.14237512269583175
0.7948757580953972 (Should be the highest score.)
0.24041798065593106


## Compute similarity

In [5]:
''' Compute similarities for all sentences. '''
def get_similar_sentences(inp, n_items=5):
    similarities = []
   
    for i in range(len(txt)):
        # store (index, score)
        similarities.append((i, inp.similarity(tokenized_txt[i])))
    
    # sort by score, return top n_items
    similarities_sorted = sorted(similarities, key=lambda item: -item[1])
    # (0 is the sentence itself.)
    return [[similarities_sorted[i][1], similarities_sorted[i][0], txt[similarities_sorted[i][0]]] for i in range(1,n_items+1)]

In [6]:
sentence = '''
Je mehr wir rechnen, desto mehr engt sich das Werden ein.
'''.replace('\n','')
sentence = spacy_de(sentence)
similar_sentences = get_similar_sentences(sentence, 8)

for score, index, sentence in similar_sentences:
    print(sentence, '\n(', score, ',', index, ')\n')

Je mehr wir uns von derlei Informationsleistungen des Kunstwerks entfernen, je sicherer können wir sein, uns innerhalb der ästhetischen Grenze zu befinden. 
( 0.8171350114467535 , 354 )

Das Selbstkonzept wird umso komplexer, je mehr Dimensionen in dem Bezugsrahmen vorliegen. 
( 0.7994579607167603 , 1507 )

Dadurch können wir beispielsweise erkennen, dass ein Problem sich nicht lösen lässt, auch wenn wir unendlich weiter rechnen. 
( 0.7958818361169535 , 863 )

Vielmehr offeriert sie unseren Gesellschaften ein Potenzial, das wir gestalten können. 
( 0.7898650538532539 , 475 )

Jeder Mensch denkt ursprünglich über das ganze Leben nach, erklärte er aber je genauer er nachdenkt, desto mehr engt sich das ein. 
( 0.7882662691450477 , 1437 )

Zunächst müssen wir uns klar darüber sein, daß das Denken, welches sich als Maschine realisiert und objektiven Status erreichen kann, ein spezifisches Denken ist, das wir operationale Theorie nennen wollen. 
( 0.7749798229541957 , 1101 )

Das Unbewusste 

/home/m/p/envs/jup/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  import sys


## Augment dataset

In [6]:
''' Get similarities for each sentence and store them as one string. '''
out = ''
# Store the amount of appearances of each sentence in a list.
import numpy as np
appearances = np.asarray([0] * len(txt))
# Loop through dataset.
n = len(txt)-1
for i in range(n):
    # Append input.
    out += txt[i] + "\n"
    # Append next sentence from original dataset.
    out += txt[i+1] + "\n"
    similar_sentences = get_similar_sentences(tokenized_txt[i])
    for score, index, sentence in similar_sentences:
        # Append input and similar sentence
        out += txt[i] + "\n"
        out += sentence + "\n"
        # Increase appearance of that sentence
        appearances[index] += 1

/home/m/p/envs/jup/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  


In [ ]:
''' Write string to disk. '''
with open('dataset_augmented.txt', 'w') as f:
    f.write(out)
    f.flush()
    f.close()

### Optional: augment and shuffle dataset

In [24]:
''' Create a list of sentence pairs (tuples). '''
out = []
# Loop through dataset.
n = len(txt)-1
for i in range(n):
    # Append input and following sentence.
    out.append((txt[i]+'\n', txt[i+1]+'\n'))
    # get similar sentences
    similar_sentences = get_similar_sentences(tokenized_txt[i])
    for score, index, sentence in similar_sentences:
        out.append((txt[i]+'\n', sentence+'\n'))

/home/m/p/envs/jup/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  


In [28]:
import random
random.shuffle(out)

In [29]:
out_str = ''
for pair in out:
    out_str += pair[0] + pair[1]

In [30]:
''' Write string to disk. '''
with open('dataset_augmented_shuffled.txt', 'w') as f:
    f.write(out_str)
    f.flush()
    f.close()

## Sources


https://medium.com/better-programming/the-beginners-guide-to-similarity-matching-using-spacy-782fc2922f7c

https://spacy.io/usage/vectors-similarity